In [4]:
import tensorflow as tf
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
if __name__ == "__main__":
    sys.path.append("../../")
from notebooks_src.box_encode_decode.configs import configs

importing Jupyter notebook from ../../notebooks_src/box_encode_decode/configs.ipynb


In [5]:
def add_epsilon(tens):
    epsilon = tf.constant(value=0.0001)
    return tf.add(tens,epsilon)
    

def average_nonzero_elements(tens):
    denominator = tf.count_nonzero(tens)
    denominator = tf.cast(denominator, tf.float32) + 0.001
    tens_sum = tf.reduce_sum(tens)
    mean_tens = tf.div(tens_sum, denominator)
    return mean_tens
    


def mask_tens(tens, mask):
    return tf.mul(tens, mask)   
    #tf.where(tf.equal(mask, tf.zeros_like(mask)),tf.zeros_like(tens),  tens)
    

def zero_out_nans(tens):
    return tf.where(condition=tf.is_nan(tens),x=tf.zeros_like(tens), y=tens)

In [6]:
def unpack_net_output(pred_tensor, feat_shape):
    ind = configs["feat_shapes"].index(feat_shape)
    sizes, ratios = configs["anchor_sizes"][ind], configs["anchor_ratios"][ind]
    num_boxes = len(sizes)+ len(ratios)
    loc_channels = num_boxes * 4
    cls_channels = configs["num_classes"] * num_boxes
    #split by grabbing the first loc_channels channels
    localizations, logits = tf.split(pred_tensor, axis=2,num_or_size_splits=[loc_channels, cls_channels])
    return localizations, logits